**Graph**:
<img src="graph.png",width="600",height="600">

** Symbol Definition **  
$x_0, x_1, ..., x_6$: car position at $0,1,...,6$ minute. Each $x_i$ is a size-2 vector (X, Y coordinates).  
$y_0^a, y_1^a, ..., y_6^a$: observations from GPS **A** at $0,1,...,6$ minute. Each $y_i^a$ is a size-2 vector.  
$y_0^b, y_1^b, ..., y_6^b$: observations from GPS **B** at $0,1,...,6$ minute. Each $y_i^b$ is a size-2 vector.  
$x_0^p$: priori knowledge of $x_0$, size-2 vector.  
Blue nodes are observed variables. White nodes are hidden variables.  


**Derivation of objective function **
1. Because there's no error associated with the car movement, the only independent variable here is $x_0$. All other $x_t$ can be derived from $x_0$.  
Car movement speed is 7.2 km/hr (0.12 km/minute), and direction is south-east, so $velocity$ vector is [0.085, -0.085].  
So $x_t = x_0 + velocity * t$. That is, $x_t[0] = x_0[0] + 0.085 * t$, $x_t[1] = x_0[1] - 0.085 * t$.    
     
2. From **Bayesian** rule, and the assumption that all observations are independent. 
$P( x_0 \, | \, x_0^p \, , \, y_0^a, \, y_0^b, ..., \, y_6^a, \, y_6^b) \propto P( x_0 \, | \, x_0^p ) \, P(y_0^a\, |\,x_0) \, P(y_0^b\, |\,x_0) ... P(y_6^a \, |\,x_0) \, P(y_6^b \, |\,x_0)$   
$P( x_0 \, | \, x_0^p ) \propto exp( - (x_0 - x_0^p) ^T B_0^{-1} (x_0 - x_0^p) )  $  
$P( y_t ^a \, | \, x_0 ) \propto exp( -(x_0 + velocity * t - y_t^a)^T R_a^{-1} (x_0 + velocity * t - y_t^a)) $  
$P( y_t ^b \, | \, x_0 ) \propto exp( -(x_0 + velocity * t - y_t^b)^T R_b^{-1} (x_0 + velocity * t - y_t^b)) $  
$B_0$ is prior convariance of $x_0$, $R_a$, $R_b$ are observation error covariances.

So the objective function to minimize is:   
$J(x_0) = (x_0 - x_0^p) ^T B_0^{-1} (x_0 - x_0^p) + \sum_{t=0}^{6} \left[ (x_0 + velocity * t - y_t^a)^T R_a^{-1} (x_0 + velocity * t - y_t^a) + (x_0 + velocity * t - y_t^b)^T R_b^{-1} (x_0 + velocity * t - y_t^b) \right] $  

** Solve with CERES **    
CERES solves objective function with following format:  
$min_x \frac{1}{2} \sum_i \rho_i (\|{f_i(x_1,x_2,...)}\|^2) $  
Here we have 15 residual blocks, i.e. 1 block for priori term ( $(x_0 - x_0^p) ^T B_0^{-1} (x_0 - x_0^p)$ ), 14 blocks for observation terms (GPS A,B, 7 times).  
For the priori term, $B_0^{-1}$ is decomposed as $B_0^{-1} = E_0^T E_0$, so that $(x_0 - x_0^p) ^T B_0^{-1} (x_0 - x_0^p) = \| E_0 (x_0 - x_0 ^p) \|^2 $. Thus, we define $f_0(x_0) = E_0 (x_0 - x_0 ^p)$.  
Similary, for observation terms, $R_a^{-1} = E_a^T E_a$, so that $(x_0 + velocity * t - y_t^a)^T R_a^{-1} (x_0 + velocity * t - y_t^a) = \| E_a (x_0 + velocity * t - y_t^a) \| ^2$.   
$R_b^{-1} = E_b^T E_b$, so that $(x_0 + velocity * t - y_t^b)^T R_b^{-1} (x_0 + velocity * t - y_t^b) = \| E_b (x_0 + velocity * t - y_t^b) \| ^2$. So the residual blocks for observations are $f_{at} = E_a (x_0 + velocity * t - y_t^a)$ and $f_{bt} = E_b (x_0 + velocity * t - y_t^b) $.  
   
See code "cartrack.cc"

** Result **   
From minimization of objective funciton, $x_0 = [ 2.126,-1.206 ]$. $x_6 = x_0 + velocity * 6 = [2.636, -1.716]$.  
Posterior covariance of $x_6$ is: $A_6 = (B_0^{-1} + 7*R_a^{-1} + 7*R_b^{-1})^{-1} = \begin{bmatrix} 
0.0207 & 0.0013 \\
0.0013 & 0.0376 \\ 
\end{bmatrix}$